In [1]:
#Cargue de las librerías
import numpy as np
import cv2
import keras
from keras import optimizers
from keras.models import *
from keras.layers import Input, Conv2D, concatenate, Add
from keras.optimizers import *
from math import pi as pi
from scipy import signal
from keras import backend as K
from math import pi as pi
import tensorflow as tf
%matplotlib qt
import matplotlib.pyplot as plt

#Función de activación cuadrática
def activation_square(x):
    return tf.square(x)

# Carga un archivo y extrae dos columnas de datos numericos kx y ky que reoresentan coordenadas espaciales en una matriz LED
# Convierte estos datos en un arreglo kxky y lo organiza como un arreglo transpuesto
def spiral_kxky(filename, ledNum):
    kxky = [[], []]
    with open(filename, 'r') as file:
        for line in file:
            for j, value in enumerate(line.split(",")):
                kxky[j].append(float(value))
    kxky = np.asarray(kxky)
    kxky = kxky.T
    return kxky[:ledNum, :]

# Recupera los pesos y los convierte en una representación de números complejos parte real e imaginaria
# Calcula la magnitud y la fase de la imagen compleja y calcula el logaritmo de esta transformación
# Muestra 6 gráficos: Magnitud y fase reconstruida,FFT, Magnitud y fase de la imagen de alta resolución orinal y FFT
def show_result(model, show=0, noShow=10):
    w_conv1 = model.get_layer('conv_O').get_weights()
    w_conv1_array = np.asarray(w_conv1)
    c_real = w_conv1_array[:, :, :, 0, :].reshape((imSize, imSize))
    c_imag = w_conv1_array[:, :, :, 1, :].reshape((imSize, imSize))
    
    c_complex = c_real + 1j * c_imag  
    c_abs = np.flip(np.flip(np.abs(c_complex), 0), 1)
    c_phase = np.flip(np.flip(np.angle(c_complex), 0), 1)
    c_complexFTLog = np.log(np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(c_complex)))))
    objFTLog = np.log(np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(obj)))))
    
    if show:
        plt.figure()
        plt.subplot(231),plt.imshow(c_abs[noShow:imSize-noShow, noShow:imSize-noShow], cmap='gray'),plt.title('recover (abs)')
        plt.subplot(232),plt.imshow(c_phase[noShow:imSize-noShow, noShow:imSize-noShow], cmap='gray'),plt.title('recover (phase)')
        plt.subplot(233),plt.imshow(c_complexFTLog[noShow:imSize-noShow, noShow:imSize-noShow], cmap='gray'),plt.title('recover FT')
        plt.subplot(234),plt.imshow(np.abs(obj[noShow:imSize-noShow, noShow:imSize-noShow]), cmap='gray'),plt.title('high res (abs)')
        plt.subplot(235),plt.imshow(np.angle(obj[noShow:imSize-noShow, noShow:imSize-noShow]), cmap='gray'),plt.title('high res (phase)')
        plt.subplot(236),plt.imshow(objFTLog[noShow:imSize-noShow, noShow:imSize-noShow], cmap='gray'),plt.title('high res FT')
        plt.show()
        
    return c_complex

In [2]:
# Se configuran los parámetros

wlength = 0.532*1e-6 # Longitud de onda de la luz verde visible
NA = 0.1 # Apertura númerica del sitema óptico
k0 = 2 * pi / wlength # Calcula el número de onda en el vacío
spsize = (3.45*1e-6)/2 # Define el tamaño del pixel de la matriz de LEDs o el tamaño del pixel del sensor
psize = spsize/4 # Establece el tamaño del pixel del sensor a una cuarta parte del tamaño del pixel de la matriz de LEDs
imSize = 128 # tamaño de la imágen a reconstruir 128 x 128 pixeles
imCenter = int(imSize / 2) # Calcula el centro de la imagen (64)
arraysize = 15 # Define el tamaño del arreglo
NAstep = 0.05 # Indica el incremento en la apertura númerica  que se utiliza para realizar calculos en diferentes ángulos o condiciones de enfoque
index_downSample = 4 # downsample: index_downSample=4 # indice de submuestro util para reducir la carga computacional

In [3]:
# Cargue de las imagenes

imgAmp = cv2.imread('cameraman.bmp', 0)+10 # Cargue de la imagen de amplitud en grises y se le suman 10 para evitar valores en 0
imgAmp = cv2.resize(imgAmp, (imSize, imSize), interpolation=cv2.INTER_CUBIC).astype(float) # Redimensionamiento de la imagen de amplitud
imgPhase = cv2.imread('westconcordorthophoto.bmp', 0) # Cargue de la imagen de la fase 
imgPhase = cv2.resize(imgPhase, (imSize, imSize), interpolation=cv2.INTER_CUBIC).astype(float) # Redimensionamiento de la imagen de fase
imgPhase = cv2.normalize(imgPhase, None, -1, 1.0, cv2.NORM_MINMAX) # se normalizan los valores entre [-1,1]
obj = imgAmp * np.exp(1j * 0.5 * pi * imgPhase) # Crea un objeto que combina la amplitud y fase en su forma compleja exponencial 

# Generación de la función de transferencia de corte

dkxy = 2*pi/psize/(imSize-1) # Calcula el paso en el espacio de frecuencias y se usa para determinar las frecuencias que frecuencias del sistema óptico puede capturar
cutoffFrequency = (NA * k0 / dkxy) # Calcula la frecuencia de corte
center = [imCenter, imCenter] # Define el centro de la imagen en el espacio de frecuencias
kYY, kXX = np.ogrid[:imSize, :imSize] # Crea 2 matrices de coordenadas para el espacio de frecuencias que contienen las posiciones de cada pixel en la imagen 
CTF = np.sqrt((kXX - center[0]) ** 2 + (kYY - center[1]) ** 2) <= cutoffFrequency # Calcula la función de transferencia de corte eb funcion de la distancia
# desde el centro de la imagen y genera una matriz binaria (True o False) si se encuentra dentro del limite de corte
CTF = CTF.astype(float) # Convierte a float 

# Se muestran las imagenes de Amplitud y Fase así como la funcion de transferencia de corte
plt.figure()
plt.subplot(1, 3, 1),plt.imshow(imgAmp, cmap='gray'),plt.title('Amplitude')
plt.subplot(1, 3, 2),plt.imshow(imgPhase, cmap='gray'),plt.title('Phase')
plt.subplot(1, 3, 3),plt.imshow(CTF, cmap='gray'),plt.title('CTF')
plt.show()

In [4]:
# Generación de imagenes de baja resolución 

imgs_train_input1 = np.ndarray((arraysize ** 2, imSize, imSize, 2)) # input real(PSF), -imag(PSF) Crea un arreglo vacio (225, 128, 128, 2)
imgs_train_input2 = np.ndarray((arraysize ** 2, imSize, imSize, 2)) # input imag(PSF), real(PSF)  Crea un arreglo vacio (225, 128, 128, 2)
kxky = spiral_kxky('spiral_kxky.txt', arraysize ** 2)   # Carga las coordenadas de frecuencia (kx,ky)
print('kxky shape:',kxky.shape) # Imprime las dimensiones del arreglo
for i in range(arraysize ** 2):
    kx = kxky[i,0] * NAstep # Extrae la componente kx de la posicion i y la multiplica por NAstep
    ky = kxky[i,1] * NAstep # Extrae la componente ky de la posicion i y la multiplica por NAstep
    kxIllu = int(kx * k0 / dkxy) # Desplazamiento en el espacio de frecuencias, utilizando la longitud de onda y el paso en el espacio de frecuencias
    kyIllu = int(ky * k0 / dkxy)
    ctfIllu = np.roll(CTF, [kxIllu, kyIllu], axis=(0, 1)) # Desplaza la mattriz CTF en las direcciones kx y ky   
    psfIllu = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(ctfIllu))) # Realiza la IFFT 2D sobre el CTF desplazado y luego aplica un desplazamiento fftshift
    psfIlluReal = np.real(psfIllu) # Extrae la parte real de la PSF iluminada
    psfIlluImag = np.imag(psfIllu) # Extrae la parte imaginaria de la PSF iluminada
    
    imgs_train_input1[i, :, :, 0] = 1 * psfIlluReal
    imgs_train_input1[i, :, :, 1] = -1 * psfIlluImag    
    imgs_train_input2[i, :, :, 0] = 1 * psfIlluImag
    imgs_train_input2[i, :, :, 1] = 1 * psfIlluReal

# Enseña los resultados    
plt.figure()
plt.subplot(1, 2, 1),plt.imshow(imgs_train_input1[0, :, :, 0], cmap='gray'),plt.title('Input real(PSF)')
plt.subplot(1, 2, 2),plt.imshow(imgs_train_input2[0, :, :, 0], cmap='gray'),plt.title('Input imag(PSF)')
plt.show()

kxky shape: (225, 2)


In [5]:
# Capas de entrada
input_1 = Input((imSize, imSize, 2), name='input_1')  # channel 1: Pr, channel 2: -Pi
input_2 = Input((imSize, imSize, 2), name='input_2')  # channel 1: Pi, channel 2: Pr

# Definición de la capa Convolucional
conv_O = Conv2D(1, imSize, activation=activation_square, padding='same', strides=index_downSample, 
                 kernel_initializer='one', bias_initializer='zero', use_bias=False, name='conv_O')
# Generación de iamgenes de baja resolución
conv1_1 = conv_O(input_1)
conv1_2 = conv_O(input_2)
addLayer = Add()([conv1_1, conv1_2]) # Una capa que combina las salidas de ambas convoluciones

model = Model(inputs=[input_1, input_2], outputs=addLayer)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_1 (InputLayer)          │ (None, 128, 128, 2)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_2 (InputLayer)          │ (None, 128, 128, 2)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv_O (Conv2D)               │ (None, 32, 32, 1)         │          32,768 │ input_1[0][0],             │
│                               │                           │                 │ input_2[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 32, 32, 1)         │               0 │ conv_O[0][0], conv_O[1][0] │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 32,768 (128.00 KB)

 Trainable params: 32,768 (128.00 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Configuración de los pesos de la capa Conv2D

weight_o = np.ndarray((1, imSize, imSize, 2, 1)) # 1 indica un solo filtro y 2 representa los canales real e imaginario
weight_o[0, :, :, 0, 0] = np.flip(np.flip(np.real(obj), 1), 0) # Se establece el primer canal como la parte real del objeto
weight_o[0, :, :, 1, 0] = np.flip(np.flip(np.imag(obj), 1), 0) # Se establece el segundo canal como la parte imaginaria del objeto
model.get_layer('conv_O').set_weights(weight_o)

# Se compila el modelo pero no se ajusta para su predicción debido a que el Lr= 0.0,por ende solo se obtienen las salidas sin actualizar los pesos
model.compile(loss='mean_absolute_error', optimizer=Adam(learning_rate=0.0))
imgs_test_predict = model.predict([imgs_train_input1, imgs_train_input2], batch_size=1, verbose=1)
plt.figure() 
plt.imshow(imgs_test_predict[0, :, :, 0],cmap='gray'),plt.title('measurement')
plt.show()

225/225 ━━━━━━━━━━━━━━━━━━━━ 88s 390ms/step


In [7]:
# Configuración de los pesos iniciales
weight_o[0, :, :, 0, 0] = np.flip(np.flip(np.sqrt(np.resize(imgs_test_predict[0, :, :, 0],(imSize,imSize))/(index_downSample**2)),1),0) # Redimensiona para asegurarse que tenga las dimensiones correctas
weight_o[0, :, :, 1, 0] = np.flip(np.flip(np.sqrt(np.resize(imgs_test_predict[0, :, :, 0],(imSize,imSize))/(index_downSample**2)),1),0) # Se calcula la raiz cuadrada de la imagen redimensionada, normalizando por el factor de downsampling
model.get_layer('conv_O').set_weights(weight_o)

# Se entrena el modelo
adam = Adam(learning_rate=1.0)
model.compile(loss='mean_absolute_error', optimizer=adam)
history = model.fit([imgs_train_input1, imgs_train_input2], imgs_test_predict, batch_size=1, epochs=10, verbose=1, shuffle=False)
imRecover = show_result(model, 1)

Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 154s 685ms/step - loss: 2882.9570
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 157s 699ms/step - loss: 2160.4844
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 159s 705ms/step - loss: 1035.9448
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 161s 713ms/step - loss: 570.2302
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 157s 699ms/step - loss: 368.0362
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 155s 688ms/step - loss: 273.1741
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 161s 717ms/step - loss: 218.0133
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 163s 723ms/step - loss: 184.8916
Epoch 9/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 166s 739ms/step - loss: 162.4454
Epoch 10/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 158s 701ms/step - loss: 152.7919
